## Import Packages

In [5]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Some Functions

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Find lane lines on the image function

In [50]:
def process_image(image):
    
    # Make image grayscale
    image_grayscale = grayscale(image)
    
    # Blur image
    image_blured = gaussian_blur(image_grayscale, 5)
    
    # Get image gradients
    image_grad = canny(image_blured, 50, 150)
    
    # Choose image region where lines are probably located
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(imshape[1]*0.45, imshape[0]*0.62), (imshape[1]*0.55, imshape[0]*0.62), (imshape[1],imshape[0])]], dtype=np.int32)
    image_roi = region_of_interest(image_grad, vertices)
    
    # Apply hough lines procedure
    rho = 1
    theta = np.pi/180
    threshold = 40
    min_line_length = 80
    max_line_gap = 150
    lines = cv2.HoughLinesP(image_roi, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)
    image_withLines = np.zeros((imshape[0], imshape[1], 3), dtype=np.uint8)
    draw_lines(image_withLines, lines)

    # Filter array with lines to get two target lines
    # make array with lines and line angles
    line_angles = np.array([[l, math.atan2(l[0][0]-l[0][2],l[0][1]-l[0][3])] for l in lines])
    # sort array by angle value
    line_angles_sorted = line_angles[line_angles[:,1].argsort()]
    # groupe lines in groups with close angle values
    line_angle_groups = []
    line_angle_group = []
    prev_angle = np.pi
    for l in line_angles_sorted:
        if abs(prev_angle - l[1]) > 0.1:
            if line_angle_group:
                line_angle_groups.append(line_angle_group)
                line_angle_group = []
        prev_angle = l[1]
        line_angle_group.append(l)
    line_angle_groups.append(line_angle_group)
    # calculate avverage line end points for each group and extend result lines to expected edges 
    lane_lines = []
    for g in line_angle_groups:
        mx1 = np.mean([l[0][0][0] for l in g])
        my1 = np.mean([l[0][0][1] for l in g])
        mx2 = np.mean([l[0][0][2] for l in g])
        my2 = np.mean([l[0][0][3] for l in g])
        if (my1 < my2):
            mt = my2; my2 = my1; my1 = mt
            mt = mx2; mx2 = mx1; mx1 = mt
        #print(mx1, my1, mx2, my2)
#        dh = (mx1-mx2)/ (my1-my2)
#        if abs(dh) > 3:
#            continue
#        my1 = imshape[0]
#        mx1 = dh*(my1-my2)+mx2
#        my2 = imshape[0]*0.65
#        mx2 = -dh*(my1-my2)+mx1
        #print(dh, mx1, my1, mx2, my2)
        lane_lines.append(np.array([[mx1, my1, mx2, my2]], dtype='int32'))
    # and draw the result!
    image_lines = np.copy(image)*0
    draw_lines(image_lines, lane_lines, thickness=10)
    image_final = weighted_img(image_lines, image)
    
    
#    result=image_grayscale
#    result=image_blured
#    result=image_grad
#    result=image_roi
#    result=image_withLines
    result=image_final
    
    return result

## Show all files in the test dir

In [33]:
import os
filesDir = "test_videos/"
allFiles = os.listdir(filesDir)
print(allFiles)
print('Total files: ', len(allFiles))

['solidYellowLeft.mp4', 'solidWhiteRight.mp4', 'challenge.mp4']
Total files:  3


## Choose file number to process

In [46]:
n = 0
try:
    fName = os.path.join(filesDir, allFiles[n]) # image file name
    print('Choosed file:', fName)
except IndexError:
    print('Please, choose file number between 0 and ', len(allFiles))

Choosed file: test_videos/challenge.mp4


In [51]:
#white_output = 'test_videos_output/solidYellowLeft.mp4'
file_output = 'test_videos_output/' + allFiles[n]
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip(fName)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(file_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4



100%|██████████| 251/251 [00:16<00:00, 15.23it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 9.41 s, sys: 874 ms, total: 10.3 s
Wall time: 18.2 s
